# Oncoming

Here we describe some new features of the developing version of MolDrug
Let's install the developing version of MolDrug. I am assuming that you already have installed RDKit and Vina dependencies. For more details in how to install MolDrug visit [how to install MolDrug](https://moldrug.readthedocs.io/en/latest/source/installation.html)

In [ ]:
#! pip install git+https://github.com/ale94mleon/MolDrug.git@main

In [14]:
import tempfile, os, requests, gzip, shutil, yaml
from multiprocessing import cpu_count
from moldrug.data import ligands, boxes, receptor_pdbqt, receptor_pdb, constraintref
tmp_path = tempfile.TemporaryDirectory()

In [15]:
# Setting the working directory (you could chang it)
wd = tmp_path.name

In [16]:
# Getting the data
lig = ligands.r_x0161
box = boxes.r_x0161
with open(os.path.join(wd, 'x0161.pdbqt'), 'w') as f:
    f.write(receptor_pdbqt.r_x0161)
with open(os.path.join(wd, 'x0161.pdb'), 'w') as f:
    f.write(receptor_pdb.r_x0161)
with open(os.path.join(wd, 'ref.sdf'), 'w') as f:
    f.write(constraintref.r_x0161)
# Getting the CReM data base
url = "http://www.qsar4u.com/files/cremdb/replacements02_sc2.db.gz"
r = requests.get(url, allow_redirects=True)
crem_dbgz_path = os.path.join(wd,'crem.db.gz')
crem_db_path = os.path.join(wd,'crem.db')
open(crem_dbgz_path, 'wb').write(r.content)
with gzip.open(crem_dbgz_path, 'rb') as f_in:
    with open(crem_db_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(lig)
print(box)
print(wd, os.listdir(wd))

COC(=O)C=1C=CC(=CC1)S(=O)(=O)N
{'A': {'boxcenter': [12.11, 1.84, 23.56], 'boxsize': [22.5, 22.5, 22.5]}}
/tmp/tmp39hmc68k ['x0161.pdb', 'ref.sdf', 'crem.db.gz', 'x0161.pdbqt', 'crem.db']


## Setting the configuration

In [17]:
config ={
    "01_grow": {
        "type": "GA",
        "njobs": 6,
        "seed_mol": lig,
        "AddHs": True,
        "costfunc": "Cost",
        "costfunc_kwargs": {
            "vina_executable": "vina",
            "receptor_pdbqt_path": os.path.join(wd, 'x0161.pdbqt'),
            "boxcenter": box['A']['boxcenter'],
            "boxsize": box['A']['boxsize'],
            "exhaustiveness": 4,
            "ncores": int(cpu_count() / 6),
            "num_modes": 1,
            "constraint": True,
            "constraint_type": "score_only", # local_only
            "constraint_ref": os.path.join(wd, 'ref.sdf'),
            "constraint_receptor_pdb_path": os.path.join(wd, 'x0161.pdb'),
            "constraint_num_conf": 100,
            "constraint_minimum_conf_rms": 0.01
        },
        "crem_db_path": crem_db_path,
        "maxiter": 5,
        "popsize": 25,
        "beta": 0.001,
        "pc": 1,
        "get_similar": False,
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 0,
            "min_inc": -5,
            "max_inc": 6,
            "ncores": 12
        },
        "save_pop_every_gen": 10,
        "deffnm": "01_grow"
    },
    "02_allow_grow": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 2,
            "min_inc": -5,
            "max_inc": 3,
            "ncores": 12
        },
        "maxiter": 5,
        "deffnm": "02_allow_grow"
    },
    "03_pure_mutate": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 1,
            "max_size": 8,
            "min_inc": -5,
            "max_inc": 3,
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "03_pure_mutate"
    },
    "04_local": {
        "mutate_crem_kwargs": {
            "radius": 3,
            "min_size": 0,
            "max_size": 1,
            "min_inc": -1,
            "max_inc": 1,
            "ncores": 12
        },
        "maxiter": 2,
        "deffnm": "04_local"
    }
}
# Save the config as a yaml file
with open(os.path.join(wd, 'config.yml'), 'w') as f:
    yaml.dump(config, f)
os.listdir(wd)

['x0161.pdb', 'ref.sdf', 'crem.db.gz', 'x0161.pdbqt', 'config.yml', 'crem.db']

## Move to the directory and run moldrug

In [19]:
cwd = os.getcwd()
os.chdir(wd)
! moldrug config.yml
os.chdir(cwd)
os.listdir(wd)
os.chdir(cwd)

You are using moldrug: 2.0.0.

The main job is being executed.


Creating the first population with 25 members:
100%|███████████████████████████████████████████| 25/25 [00:16<00:00,  1.50it/s]
Initial Population: Best individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0)
File 01_grow_pop.sdf was createad!
Evaluating generation 1 / 5:
100%|███████████████████████████████████████████| 24/24 [00:22<00:00,  1.04it/s]
Generation 1: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 2 / 5:
100%|███████████████████████████████████████████| 25/25 [00:25<00:00,  1.04s/it]
Generation 2: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 3 / 5:
100%|███████████████████████████████████████████| 25/25 [00:30<00:00,  1.23s/it]
Generation 3: Best Individual: Individual(idx = 0, smiles = COC(=O)c1ccc(S(N)(=O)=O)cc1, cost = 1.0).

Evaluating generation 4 / 

In [21]:
os.listdir(wd)

['x0161.pdb',
 '01_grow_pop.sdf',
 '04_local_result.pbz2',
 'ref.sdf',
 '02_allow_grow_pop.pbz2',
 '03_pure_mutate_result.pbz2',
 'crem.db.gz',
 '03_pure_mutate_pop.sdf',
 '04_local_pop.pbz2',
 '01_grow_result.pbz2',
 '02_allow_grow_pop.sdf',
 '03_pure_mutate_pop.pbz2',
 '02_allow_grow_result.pbz2',
 '04_local_pop.sdf',
 'x0161.pdbqt',
 'config.yml',
 'crem.db',
 '01_grow_pop.pbz2']

In [20]:
import nglview as nv
import parmed as pmd
from ipywidgets import IntSlider, VBox

In [30]:
parms = pmd.rdkit.from_sdf(os.path.join(wd, '04_local_pop.sdf')) # require parmed, rdkit
parms

[<Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 28 atoms; 1 residues; 28 bonds; NOT parametrized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT parametrized>,
 <Structure 23 atoms; 1 residues; 23 bonds; NOT parametrized>,
 <Structure 24 atoms; 1 residues; 24 bonds; NOT parametrized>,
 <Structure 27 atoms; 1 residues; 27 bonds; NOT parametrized>,
 <Structure 32 atoms; 1 residues; 32 bonds; NOT parametrized>,
 <Structure 30 atoms; 1 residues; 30 bonds; NOT parametrized>,
 <Structure 37 atoms; 1 residues; 37 bonds; NOT parametrized>,
 <Structure 30 atoms; 1 residues; 30 bonds; NOT parametrized>,
 <Structure 31 atoms; 1 residues; 31 bonds; NOT parametrized>,
 <Structure 25 atoms; 1 residues; 25 bonds; NOT parametrized>,
 <Structure 34 atoms; 1 residues; 34 bonds; NOT parametrized>,
 <Structure 26 atoms; 1 residues; 26 bonds; NOT parametrized>,
 <Structure 26 atoms; 1 residues; 26 bonds; NOT parametrized>,
 <Structure 23 atoms; 1 residues; 23 bonds; NOT paramet

In [31]:
view = nv.NGLWidget()

slider = IntSlider(max=len(parms)-1)

def show_one_ligand(change):
    val = change['new']
    view.show_only([val])
    
slider.observe(show_one_ligand, 'value')

VBox([view, slider])

In [32]:
for p in parms:
    view.add_structure(nv.ParmEdTrajectory(p))
view.show_only([0])

In [ ]:

# [(view[i].clear(), view[i].add_licorice()) for i in range(4)]